In [2]:
import sklearn
import numpy as np
import pandas as pd
df = pd.read_csv('nlp dataset.csv')
df.head()


,Unnamed: 0,sentence,label
0,0,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,sentence
1,1,"Born and raised in Houston, Texas, she perform...",sentence
2,2,"Managed by her father, Mathew Knowles, the gro...",sentence
3,3,Their hiatus saw the release of Beyoncé's debu...,sentence
4,4,Following the disbandment of Destiny's Child i...,sentence


In [3]:
import re
from pprint import pprint
import nltk, spacy, gensim
from sklearn.feature_extraction.text import CountVectorizer
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [4]:
data_lemmatized = df.sentence.apply(preprocess_text)


In [5]:
data_lemmatized

0         Beyonc Giselle Knowles Carter bi nse bee YON s...
1         Born and raised in Houston Texas she performed...
2         Managed by her father Mathew Knowles the group...
3         Their hiatus saw the release of Beyonc debut a...
4         Following the disbandment of Destiny Child in ...
                                ...                        
235105                                        hi teensUser 
235106                                                 JOIN
235107                                        Hi teensUser 
235108                          Not that know of teensUser 
235109                                                  Uh 
Name: sentence, Length: 235110, dtype: object

In [6]:
from sklearn.model_selection import train_test_split
sentences_train, sentences_test, y_train, y_test = train_test_split(data_lemmatized,df.label, test_size=0.25, random_state=1000)

In [7]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test  = vectorizer.transform(sentences_test)




# Tfidf

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(sentences_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(sentences_test)
print(X_train_vectors_tfidf)


  (0, 39651)	0.18213474082870504
  (0, 59547)	0.16700603471983885
  (0, 2570)	0.062621848222198
  (0, 48995)	0.21961724312437506
  (0, 71263)	0.20123977067131815
  (0, 63943)	0.0389928684480599
  (0, 45273)	0.09222461088152092
  (0, 52598)	0.2381105231428315
  (0, 20152)	0.30811122009848424
  (0, 9092)	0.1276298731310181
  (0, 51763)	0.1923255030139485
  (0, 28998)	0.149516248463318
  (0, 44981)	0.04962800771497838
  (0, 6052)	0.10618822363669338
  (0, 45309)	0.1368834238887918
  (0, 44299)	0.11263197850773331
  (0, 42435)	0.3442078432766714
  (0, 70493)	0.15180738342315622
  (0, 3989)	0.19298424170951936
  (0, 21217)	0.17422548664526105
  (0, 17590)	0.1940042984667248
  (0, 63931)	0.09141598571349249
  (0, 4225)	0.24615677240712125
  (0, 63941)	0.30223179467055905
  (0, 34105)	0.31569111780097064
  :	:
  (176330, 32170)	0.23848437558390165
  (176330, 15931)	0.23320174389324014
  (176330, 33661)	0.2692269538371488
  (176330, 44874)	0.2892622530870145
  (176330, 45285)	0.181096045097164

# LSA

In [9]:
from sklearn.decomposition import TruncatedSVD
lsa_obj = TruncatedSVD(n_components=20, n_iter=100, random_state=42)
tfidf_lsa_data = lsa_obj.fit_transform(X_train_vectors_tfidf)
Sigma = lsa_obj.singular_values_
V_T = lsa_obj.components_.T
V_T



array([[ 6.90336098e-04, -7.38972246e-05, -4.59608076e-04, ...,
         3.90531149e-04, -6.61622245e-05, -3.66857348e-04],
       [ 1.51037036e-04, -1.77582815e-05, -1.85547242e-04, ...,
         3.76893272e-04, -1.03863678e-04,  5.45549359e-05],
       [-1.85831754e-31, -1.30450429e-21, -2.85028635e-19, ...,
        -4.16065729e-17,  8.65419944e-17, -4.44517406e-17],
       ...,
       [ 4.22525435e-05, -2.05869732e-05,  1.49376524e-04, ...,
        -1.64320139e-04,  4.92998533e-05, -1.14395874e-05],
       [ 9.92078085e-06, -3.34809920e-06, -1.17328649e-05, ...,
         7.30818492e-05,  2.91996033e-04,  8.26666698e-06],
       [ 1.27995435e-07, -2.64060100e-08, -3.48594650e-07, ...,
        -6.13941414e-07,  3.78206085e-06,  2.73265151e-07]])

# LDA Model

In [10]:
FILTER_REGEX = re.compile(r"[a-zA-Z0-9]{3,}")

def tokenize_line(line):
    res = line.lower().split()
    tokens=[]
    for word in res:
        if re.findall(FILTER_REGEX,word):
            tokens.append(word)
    return tokens
    
def tokenize(lines):
    tokens = [tokenize_line(line) for line in lines]
    return tokens
    
    
d = [one for one in tokenize(data_lemmatized)]

In [11]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

from gensim.corpora import Dictionary
dictionary = Dictionary(d)
stopwords = nltk_stopwords()
stopword_ids = map(dictionary.token2id.get, stopwords)
dictionary.filter_tokens(stopword_ids)
dictionary.compactify()
dictionary.filter_extremes(no_below=10, no_above=1, keep_n=None)
dictionary.compactify()
corpus = [dictionary.doc2bow(doc) for doc in d]


In [12]:
from gensim import models
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=10)

In [13]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
vis_data = gensimvis.prepare(lda, corpus, dictionary, mds='tsne')
panel = pyLDAvis.display(vis_data)
panel

# sentence embedding

In [14]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_sentences, test_sentences = train_test_split(data_lemmatized, test_size=0.2, random_state=42)

# Tag the sentences
tagged_train_sentences = [TaggedDocument(words=sent.split(), tags=[i]) for i, sent in enumerate(train_sentences)]
tagged_test_sentences = [TaggedDocument(words=sent.split(), tags=[i]) for i, sent in enumerate(test_sentences)]

# Initialize the model
model = Doc2Vec(vector_size=50, min_count=2, epochs=40)

# Build the vocabulary
model.build_vocab(tagged_train_sentences)

# Train the model
model.train(tagged_train_sentences, total_examples=model.corpus_count, epochs=model.epochs)

# Now you can use the trained model to convert sentences into vectors
train_vectors = [model.infer_vector(sent.split()) for sent in train_sentences]
test_vectors = [model.infer_vector(sent.split()) for sent in test_sentences]

# word embedding

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences_train)

X_train_t = tokenizer.texts_to_sequences(sentences_train)
X_test_t = tokenizer.texts_to_sequences(sentences_test)



vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
maxlen = 250  # Fixed the maxlen value

X_train_t = pad_sequences(X_train_t, padding='post', maxlen=maxlen)
X_test_t = pad_sequences(X_test_t, padding='post', maxlen=maxlen)
encoder = LabelEncoder()
encoder.fit(y_train)
y_test_t = encoder.fit_transform(y_test)
y_train_t = encoder.fit_transform(y_train)



print("Shape after tokenization (X_train_t):", X_train_t.shape)


# Assuming that `y_test` is your list of labels




Shape after tokenization (X_train_t): (176332, 250)


In [16]:
y_test_t

array([1, 0, 0, ..., 0, 0, 1])

# Logistic regression

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

lr_pipeline = Pipeline([
    ('vect', TfidfVectorizer(stop_words='english', ngram_range=(1, 2))),
    ('clf', LogisticRegression(C=1.0, class_weight='balanced')),
])

lr_pipeline.fit(sentences_train, y_train)
print(lr_pipeline.score(sentences_test, y_test),lr_pipeline.predict(["this is good"]))




0.7768722991595495 ['sentence']


# Random forest classifier

In [18]:
from sklearn.ensemble import RandomForestClassifier
rf_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', RandomForestClassifier(max_depth=15, n_estimators=40)),
])
rf_pipeline.fit(sentences_train, y_train)
print(rf_pipeline.score(sentences_test, y_test),rf_pipeline.predict(["is this good?"]))


0.7558270101058219 ['question']


# Decision Tree Classifier

In [19]:
from sklearn.tree import DecisionTreeClassifier
dt_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', DecisionTreeClassifier(max_depth=25)),
])
dt_pipeline.fit(sentences_train, y_train)
print(dt_pipeline.score(sentences_test, y_test),dt_pipeline.predict(["this is good"])
)

0.9659396372792541 ['sentence']


# SVM 

In [21]:
from sklearn.svm import SVC
svm_pipeline = Pipeline([
    ('vect', CountVectorizer()),
    ('clf', SVC(kernel='rbf', C=1.0, gamma='scale')),
])
svm_pipeline.fit(sentences_train, y_train)
print(svm_pipeline.score(sentences_test, y_test),svm_pipeline.predict(["this is good"]))


0.9819150022117119 ['sentence']


# plot curve (please run it once because i made changes in this but did not have enough time to run it since begining. the code gives the graph, kindly run it once)

In [2]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt


y_pred_dt = dt_pipeline.predict_proba(sentences_test)[:, 1]
fpr_dt, tpr_dt, thresholds_dt = roc_curve(y_test, y_pred_dt,pos_label="sentence")
auc_dt = auc(fpr_dt, tpr_dt)

y_pred_rf = rf_pipeline.predict_proba(sentences_test)[:, 1]
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_pred_rf,pos_label="sentence")
auc_rf = auc(fpr_rf, tpr_rf)

y_pred_lr = lr_pipeline.predict_proba(sentences_test)[:, 1]
fpr_lr, tpr_lr, thresholds_lr = roc_curve(y_test, y_pred_lr,pos_label="sentence")
auc_lr = auc(fpr_lr, tpr_lr)


y_pred_svc = svm_pipeline.decision_function(sentences_test)
fpr_svc, tpr_svc, thresholds_svc = roc_curve(y_test, y_pred_svc,pos_label="sentence")
auc_svc = auc(fpr_lr, tpr_lr)




plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_svc, tpr_svc, label='svc (area = {:.3f})'.format(auc_svc))
plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.plot(fpr_lr, tpr_lr, label='LR (area = {:.3f})'.format(auc_lr))
plt.plot(fpr_dt, tpr_dt, label='DT (area = {:.3f})'.format(auc_dt))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()



In [23]:
import os

# Get the current directory
current_dir = os.getcwd()

# List all files in the current directory
files = os.listdir(current_dir)

print("Current directory:", current_dir)
print("Files in current directory:", files)

Current directory: c:\Users\manoj\Downloads\Ml
Files in current directory: ['Assignment 4_ML.ipynb', 'desktop.ini', 'Dockerfile', 'nlp dataset.csv']


In [24]:

from tensorflow.keras.models import save_model
import joblib
import pickle

# Specify the path to the directory where you want to save the models
dir_path = 'C:\\Users\\manoj\\Downloads'

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)


with open(os.path.join(dir_path, 'lr_pipeline.pkl'), 'wb') as f:
    pickle.dump(lr_pipeline, f)

with open(os.path.join(dir_path, 'dt_pipeline.pkl'), 'wb') as f:
    pickle.dump(dt_pipeline, f)

with open(os.path.join(dir_path, 'rf_pipeline.pkl'), 'wb') as f:
    pickle.dump(rf_pipeline, f)

# with open(os.path.join(dir_path, 'svc.pkl'), 'wb') as f:
    # pickle.dump(svm_pipeline, f)


In [25]:
# Specify the path to the directory where you want to save the models
dir_path = 'C:\\Users\\manoj\\Downloads'

# Create the directory if it doesn't exist
os.makedirs(dir_path, exist_ok=True)

with open(os.path.join(dir_path, 'vectorizer.pkl'), 'wb') as f:
    pickle.dump(vectorizer, f)

with open(os.path.join(dir_path, 'tfidf_vectorizer.pkl'), 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)



In [26]:
from flask import Flask, request, jsonify
import requests
import json
import torch
from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from gensim import models
from sklearn.decomposition import TruncatedSVD

app = Flask(__name__)


lr_pipeline = joblib.load('C:\\Users\\manoj\\Downloads\\lr_pipeline.pkl')
dt_pipeline = joblib.load('C:\\Users\\manoj\\Downloads\\dt_pipeline.pkl')
rf_pipeline = joblib.load('C:\\Users\\manoj\\Downloads\\rf_pipeline.pkl')
# svm_pipeline= joblib.load('C:\\Users\\manoj\\Downloads\\svm_pipeline.pkl')

# Load your vectorizers
vectorizer = joblib.load('C:\\Users\\manoj\\Downloads\\vectorizer.pkl')
tfidf_vectorizer = joblib.load('C:\\Users\\manoj\\Downloads\\tfidf_vectorizer.pkl')


@app.route('/')
def home():
    return "Hello, World!"

@app.route('/predict', methods=['POST'])


def predict():
    data = request.get_json(force=True)
    text = data['text']
    model = data.get('model', 'lr') 
    # model = data.get('model', 'lr')

    # Vectorize the input text
    text_vector = vectorizer.transform([text])
    text_tfidf = tfidf_vectorizer.transform([text])
    text_lsa = lsa_obj.transform(text_tfidf)
    text_lda = lda[text_vector]

    # Make predictions using your models
    if model == 'lr':
        prediction = lr_pipeline.predict(text_vector)
    elif model == 'dt':
        prediction = dt_pipeline.predict(text_vector)
    elif model == 'rf':
        prediction = rf_pipeline.predict(text_vector)
    elif model == 'svc':
        prediction = svm_pipeline.predict(text_vector)
    else:
        return jsonify({'error': 'Invalid model specified'}), 400

    # Return the predictions as JSON
    return jsonify({
        'prediction': prediction.tolist()
    })

if __name__ == '__main__':
    app.run()




 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
